In [82]:
def my_apriori(matrix, minsup):
    
    def prune_itemsets(candidates, previous_frequent, minsup, supports):
        # 1st pruning: remove all the itemsets containing one or more infrequent subsets
        to_remove = []
        for i in range(len(candidates)):
            for j in range(len(candidates[i])):
                subset = [candidates[i][k] for k in range(len(candidates[i])) if k != j]
                if(subset not in previous_frequent):
                    to_remove.append(i)
                    break
        candidates = [candidates[i] for i in range(len(candidates)) if i not in to_remove]
        
        # 2nd pruning: remove all itemsets whose support < minsup
        to_remove = []
        for i in range(len(candidates)):
            combined_array = np.array([1 for i in range(matrix.shape[0])])
            for j in range(len(candidates[i])):
                combined_array = combined_array & matrix[:,candidates[i][j]]
            support = sum(combined_array)
            if(support < minsup):
                to_remove.append(i)
            else:
                supports += [([candidates[i][j] for j in range(len(candidates[i]))], support/matrix.shape[0])]
        candidates = [candidates[i] for i in range(len(candidates)) if i not in to_remove]
        
        return candidates

    def generate_couples(v):
        couples = []
        for i in range(len(v)):
            for j in range(i + 1, len(v)):
                couples.append([v[i][0], v[j][0]])
        return couples 

    def generate_candidates(frequent, k):
        candidates = []
        for i in range(len(frequent)):
            for j in range(i + 1,len(frequent)):
                if frequent[i][:k-1] == frequent_itemsets[j][:k-1]:
                    candidates.append(frequent[i][:k] + [frequent[j][k - 1]])
        return candidates

    
    minsup = minsup*matrix.shape[0]
    supports = []
    frequent_itemsets = [[i] for i in range(matrix.shape[1]) if sum(matrix[:,i]) >= minsup]
    supports += [(i, sum(matrix[:,i[0]])/matrix.shape[0]) for i in frequent_itemsets]
    
    k = 1
    while frequent_itemsets != [] and k < matrix.shape[1]:
        if k == 1:
            candidate_itemsets = generate_couples(frequent_itemsets)
        else:
            candidate_itemsets = generate_candidates(frequent_itemsets, k)               
        frequent_itemsets = prune_itemsets(candidate_itemsets, frequent_itemsets, minsup, supports)
        k += 1
        
    supports.sort(key= lambda tup: tup[1], reverse = True)
    return supports
    

In [83]:
import json

with open("modified_coco.json") as f: 
    obj = json.load(f)

transactions = []
distinct_items = []
for i in range(len(obj)):
    new_transaction = list(set(obj[i]["annotations"]))
    transactions.append(new_transaction)
    for j in range(len(new_transaction)):
        if new_transaction[j] not in distinct_items:
            distinct_items.append(new_transaction[j])
            

In [84]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
import timeit

matrix = []
for i in range(len(transactions)):
    presence_vector = [(x in transactions[i]) for x in distinct_items]
    matrix.append(presence_vector)
matrix = np.array(matrix)    

df = pd.DataFrame(data=matrix, columns=distinct_items)
minsup = 0.05

In [85]:
my_supports = my_apriori(matrix, minsup)
supports = apriori(df, min_support=minsup)

In [75]:
t1 = timeit.timeit(lambda: my_apriori(matrix, minsup), number = 1)
t2 = timeit.timeit(lambda: apriori(df, min_support=minsup), number = 1) 

time_comp = t1/t2

if time_comp >= 1:
    print(f"Your implementation is {time_comp:.2f} times slower than the pre-implemented one.")
else:
    print(f"Your implementation is {1/time_comp:.2f} times faster than the pre-implemented one.")
        

Your implementation is 99.46 times slower than the pre-implemented one.


In [86]:
print("\tsupport\titemsets")
for i in range(len(my_supports)):
    print(f"{i}\t{my_supports[i][1]}\t{[distinct_items[j] for j in my_supports[i][0]]}")

print(supports)  

	support	itemsets
0	0.5886	['person']
1	0.4338	['bench']
2	0.3704	['car']
3	0.323	['traffic light']
4	0.3208	['bench', 'person']
5	0.2386	['car', 'person']
6	0.1978	['car', 'traffic light']
7	0.1902	['person', 'traffic light']
8	0.1362	['car', 'person', 'traffic light']
9	0.1346	['fire hydrant']
10	0.1332	['stop sign']
11	0.1286	['truck']
12	0.123	['handbag']
13	0.1224	['person', 'handbag']
14	0.1032	['car', 'truck']
15	0.0912	['bus']
16	0.0852	['backpack']
17	0.0828	['person', 'truck']
18	0.0826	['person', 'backpack']
19	0.0778	['truck', 'traffic light']
20	0.0762	['bicycle']
21	0.0762	['person', 'bus']
22	0.0712	['car', 'bench']
23	0.0668	['bus', 'traffic light']
24	0.0666	['person', 'bicycle']
25	0.066	['car', 'truck', 'traffic light']
26	0.0656	['car', 'bus']
27	0.0656	['car', 'person', 'truck']
28	0.0622	['car', 'fire hydrant']
29	0.0604	['car', 'bench', 'person']
30	0.0602	['chair']
31	0.0584	['bench', 'handbag']
32	0.058	['bench', 'person', 'handbag']
33	0.0576	['stop sign', 'ca